<a href="https://colab.research.google.com/github/Ram-N/2019-ICC-worldcup-prediction-contest/blob/master/SA_4_5_Concatenating_Multiple_Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  ## Pandas Full Example of Data Preparation



  # Reading Multiple CSV files, and Concatenating them

  <img src = "../img/sa_logo.png" width="100" align="left">

  Ram Narasimhan

  <br><br><br>



Concepts covered in this notebook.

1. A function to process a single df
2. Loop over all Years, and store each df in a dictionary
3. pd.concat() all the years into one long `all_seasons_df`
4. Subset the big df to get data on any coach

In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


### 1. A function to process a single df



In [42]:
def prepare_single_df(df, curr_season):
  '''
  Takes in a df, and returns cleaned up version with proper column names and a season column

  A number of renaming steps are done.
  For each Season, we create a new Column and store that Year

  '''
  df.columns = df.columns.str.replace('w/ ','with_')
  df.columns = df.columns.str.replace('Team','Team_')
  df.columns = df.columns.str.replace('.','-', regex=False)
  df.columns = df.columns.str.replace(' ','_')
  column_categories = df.columns
  # Make the first row the column names
  df.columns = df.iloc[0]
  # Drop the first row
  df = df[1:].reset_index(drop=True)

  df.rename(columns={'Tm':'Team'}, inplace=True)

  current_cat = ""
  new_column_names = []
  for idx, cat in enumerate(column_categories):
    if "Unnamed" not in cat: #we have found a new category
      current_cat = cat + "_"
    if "Season" in cat:
      current_cat = 'Season_' # we don't want the Year. We will make a separate column for it.
    new_column_names.append(current_cat + df.columns[idx])

  df.columns = new_column_names
  df.columns = df.columns.str.replace('.','-', regex=False)
  df.columns = df.columns.str.replace(' ','_', regex=False)

  remark_columns = df.filter(like='Remark')

  if not remark_columns.empty:
      # Get the first matching column and rename it
      column_to_rename = remark_columns.columns[0]
      df.rename(columns={column_to_rename: 'Remark'}, inplace=True)


  for column in df.columns:
    if '_plyf' in column:
      new_column = 'Playoff_' + column
      df.rename(columns={column: new_column}, inplace=True)

  df.columns = df.columns.str.replace("_plyf", "") #drop the _plyf
  df.columns = df.columns.str.replace("Playoff_Playoffs", "Playoff_") # take care of dupes
  df.columns = df.columns.str.replace("__", "_", regex=False) #clean up all dunders to single underscore

  #create a new column to store the season along with the row
  df['Season'] = curr_season

  #rearrange columns
  cols = df.columns.tolist()
  new_order = cols[:2] + [cols[-1]] + cols[2:-1]
  df = df[new_order]

  return df

We are going to create an empty dictionary, and start storing the `curr_season` as the key, and each year's Coach standings `df` as the "value"

### 2. Loop over all Years, and store each df in a dictionary


In [43]:
season = {}

for curr_season in range(2000, 2023):
  print(f'Processing ... {curr_season}')
  url = f'https://raw.githubusercontent.com/keshavmp/Sports_Analytics_Project/main/{curr_season}_Coach_Standings.csv'
  df = pd.read_csv(url)
  df = prepare_single_df(df, curr_season)

  season[curr_season] = df
  print(f'--------'*5)

Processing ... 2000
----------------------------------------
Processing ... 2001
----------------------------------------
Processing ... 2002
----------------------------------------
Processing ... 2003
----------------------------------------
Processing ... 2004
----------------------------------------
Processing ... 2005
----------------------------------------
Processing ... 2006
----------------------------------------
Processing ... 2007
----------------------------------------
Processing ... 2008
----------------------------------------
Processing ... 2009
----------------------------------------
Processing ... 2010
----------------------------------------
Processing ... 2011
----------------------------------------
Processing ... 2012
----------------------------------------
Processing ... 2013
----------------------------------------
Processing ... 2014
----------------------------------------
Processing ... 2015
----------------------------------------
Processing ... 2016
----

In [44]:
season[2017].sample()

,Coach,Team,Season,Season_G,Season_W,Season_L,Season_T,with_Team_G,with_Team_W,with_Team_L,...,Playoff_G,Playoff_W,Playoff_L,Playoff_with_Team_-1_G,Playoff_with_Team_-1_W,Playoff_with_Team_-1_L,Playoff_Career-1_G,Playoff_Career-1_W,Playoff_Career-1_L,Remark
23,Ben McAdoo,NYG,2017,12,2,10,0,28,13,15,...,NaN,NaN,NaN,1,0,1,1,0,1,Fired 12/04


### 3. pd.concat() all the years into one long `all_seasons_df`


In [45]:
# Concatenate DataFrames vertically
all_seasons_df = pd.concat(season.values(), axis=0, ignore_index=True)

# Display the concatenated DataFrame
print(all_seasons_df.shape)

(778, 25)


In [46]:
# Convert all columns to int and handle errors
all_seasons_df = all_seasons_df.apply(pd.to_numeric, errors='ignore', downcast='integer')

In [47]:
all_seasons_df.dtypes

Coach                      object
Team                       object
Season                      int16
Season_G                     int8
Season_W                     int8
Season_L                     int8
Season_T                     int8
with_Team_G                 int16
with_Team_W                 int16
with_Team_L                  int8
with_Team_T                  int8
Career_G                    int16
Career_W                    int16
Career_L                    int16
Career_T                     int8
Playoff_G                 float64
Playoff_W                 float64
Playoff_L                 float64
Playoff_with_Team_-1_G    float64
Playoff_with_Team_-1_W    float64
Playoff_with_Team_-1_L    float64
Playoff_Career-1_G        float64
Playoff_Career-1_W        float64
Playoff_Career-1_L        float64
Remark                     object
dtype: object

### 4. Subset the big df to get data on any coach

Now that we have a nice long `all_seasons_df` we can pull the rows for any one coach, or one season.

Let's try to find all the rows for the Coach with the most PlayOff Career wins.

In [48]:
max_id = all_seasons_df['Playoff_Career-1_W'].idxmax() # Row ID of the coach with most playoff career wins
coach = all_seasons_df.loc[max_id]['Coach'] # Get that coach's name

Filter just the columns for that Coach.

In [49]:
all_seasons_df[all_seasons_df['Coach']==coach]

,Coach,Team,Season,Season_G,Season_W,Season_L,Season_T,with_Team_G,with_Team_W,with_Team_L,...,Playoff_G,Playoff_W,Playoff_L,Playoff_with_Team_-1_G,Playoff_with_Team_-1_W,Playoff_with_Team_-1_L,Playoff_Career-1_G,Playoff_Career-1_W,Playoff_Career-1_L,Remark
21,Bill Belichick,NWE,2000,16,5,11,0,386,266,120,...,0.0,0.0,0.0,42.0,30.0,12.0,44.0,31.0,13.0,NaN
54,Bill Belichick,NWE,2001,16,11,5,0,386,266,120,...,3.0,3.0,0.0,42.0,30.0,12.0,44.0,31.0,13.0,Super Bowl Champions
86,Bill Belichick,NWE,2002,16,9,7,0,386,266,120,...,0.0,0.0,0.0,42.0,30.0,12.0,44.0,31.0,13.0,NaN
119,Bill Belichick,NWE,2003,16,14,2,0,386,266,120,...,3.0,3.0,0.0,42.0,30.0,12.0,44.0,31.0,13.0,Super Bowl Champions
153,Bill Belichick,NWE,2004,16,14,2,0,386,266,120,...,3.0,3.0,0.0,42.0,30.0,12.0,44.0,31.0,13.0,Super Bowl Champions
186,Bill Belichick,NWE,2005,16,10,6,0,386,266,120,...,2.0,1.0,1.0,42.0,30.0,12.0,44.0,31.0,13.0,NaN
219,Bill Belichick,NWE,2006,16,12,4,0,386,266,120,...,3.0,2.0,1.0,42.0,30.0,12.0,44.0,31.0,13.0,NaN
252,Bill Belichick,NWE,2007,16,16,0,0,386,266,120,...,3.0,2.0,1.0,42.0,30.0,12.0,44.0,31.0,13.0,AFC Champions
284,Bill Belichick,NWE,2008,16,11,5,0,386,266,120,...,0.0,0.0,0.0,42.0,30.0,12.0,44.0,31.0,13.0,NaN
320,Bill Belichick,NWE,2009,16,10,6,0,386,266,120,...,1.0,0.0,1.0,42.0,30.0,12.0,44.0,31.0,13.0,NaN


This notebook showed us a number of ways to work with Pandas. We used a for loop to read multiple files in one go.

We stored each of those dataframes in a dict with the year as the key. (This allows us to access any season's standings easily.)

Finally, we used `pd.concat()` to vertically stack all the dfs into one "tall" dataframe.

This tall df (`all_seasons_df`) allows us to filter for any coach(es) of interest.